In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.metrics import log_loss
from sklearn.preprocessing import KBinsDiscretizer
import category_encoders as ce
from tpot import TPOTClassifier
import timeit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from tpot.builtins import ZeroCount
from sklearn.preprocessing import RobustScaler

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv', sep=',')
train = train.set_index('id')

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv', sep=',')
test = test.set_index('id')

In [ ]:
fig = px.histogram(train, x="target",
                   width=600, 
                   height=400,
                   histnorm='percent',
                   template="simple_white"
                   )
fig.update_layout(title="Target Description", 
                  font_family="San Serif",
                  titlefont={'size': 20},
                  showlegend=True,
                  legend=dict(
                      orientation="v", 
                      y=1, 
                      yanchor="top", 
                      x=1.0, 
                      xanchor="right"
                  )                
                 ).update_xaxes(categoryorder='total descending')#
fig.update_traces( 
                  marker_line_width=1.5, opacity=0.99)
fig.show()

In [ ]:
cols = train.columns[:-1] 
df1= train[train['target']==train.target.unique()[0]]
df2= train[train['target']==train.target.unique()[1]]
df3= train[train['target']==train.target.unique()[2]]
df4= train[train['target']==train.target.unique()[3]]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=df1[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df2[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[1]),
                         line=dict(color='red', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df3[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[2]),
                         line=dict(color='orange', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[3]),
                         line=dict(color='yellow', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[4]),
                         line=dict(color='lightblue', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[5]),
                         line=dict(color='grey', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[6]),
                         line=dict(color='violet', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[7]),
                         line=dict(color='green', width=2, dash='solid')
                         ))


fig.update_layout(title='<b>Average value by Class<b>',
                  xaxis_title='Features',
                  yaxis_title='Average Value',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)

fig.update_yaxes(title_text="<b>values</b>", secondary_y=True)

fig.show()    


In [ ]:
train['sum_row'] = train[cols].sum(axis=1)
fig = px.box(train,x='target', y="sum_row")

fig.update_layout(title='<b>Sum row wise by class<b>',
                  xaxis_title='Class',
                  yaxis_title='Sum of features',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)

fig.show()
train = train.drop(columns=['sum_row'])

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(train[cols]<0).sum(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>#Negative istances by feature in training set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(test[cols]<0).sum(),
                         x=cols,
                         name = '#negative',
                         line=dict(color='green', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>#Negative istances by feature in test set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
df = pd.concat([train[cols], test[cols]], axis=0)

In [ ]:
unique_df = pd.DataFrame(df.nunique()).reset_index()
unique_df.columns=['features','count']

fig1 = px.bar(unique_df, y='count', x=cols)

fig1.update_layout(title='Feature cardinality in train+test set',
                  xaxis_title='features',
                  yaxis_title='# unique values',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1500, height=500)
fig1.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(train[cols]==0).mean(),
                         x=cols,
                         name = 'train_features=0',
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>% features=0 in training Set set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(test[cols]==0).mean(),
                         x=cols,
                         name = 'test_features=0',
                         line=dict(color='green', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>% features=0 in test Set set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

features are more correlated in this competition. While in May TPS I set max and min euqal to 0.05

In [ ]:
corr = train[train.columns[:-2]].corr()
fig = go.Figure(data= go.Heatmap(z=corr,
                                 x=corr.index.values,
                                 y=corr.columns.values,
                                 zmin=-0.3,
                                 zmax=0.3
                                 )
                )
fig.update_layout(title_text='<b>Correlation Matrix<b>',
                  title_x=0.5,
                  titlefont={'size': 24},
                  width=900, height=800,
                  xaxis_showgrid=False,
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed', 
                  paper_bgcolor=None,
                  )
fig.show()

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv', sep=',')
train = train.set_index('id')
test= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv', sep=',')
test = test.set_index('id')

In [ ]:
cols = train.columns[:-1]
target = train['target']
train = train[cols]

## MLJAR multiclass Classifier

In [ ]:
!pip install -q -U matplotlib==3.1.3

In [ ]:
!pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

In [ ]:
from supervised.automl import AutoML

In [ ]:
automl = AutoML(
    mode = "Compete",
    algorithms=["CatBoost", "Xgboost"],
    total_time_limit=7*3600,
    start_random_models=10,
    hill_climbing_steps=3,
    top_models_to_improve=3,
    golden_features=False,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=1,
    ml_task = 'multiclass_classification',
    eval_metric='logloss',
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 5,
        "shuffle": False,
        "stratify": True,
    }
)

In [ ]:
automl.fit(train, target)

In [ ]:
automl.report()

### Load Submission Sample

In [ ]:
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv', sep=',')
sub_sample = sub_sample.set_index('id')

### Predictions

In [ ]:
predictions = automl.predict_all(test)

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv', sep=',')
test = test.set_index('id')

In [ ]:
pred= predictions.set_index(test.index)

In [ ]:
pred.head()

In [ ]:
sub_sample['Class_1'] = pred['prediction_Class_1']
sub_sample['Class_2'] = pred['prediction_Class_2']
sub_sample['Class_3'] = pred['prediction_Class_3']
sub_sample['Class_4'] = pred['prediction_Class_4']
sub_sample['Class_5'] = pred['prediction_Class_5']
sub_sample['Class_6'] = pred['prediction_Class_6']
sub_sample['Class_7'] = pred['prediction_Class_7']
sub_sample['Class_8'] = pred['prediction_Class_8']
sub_sample['Class_9'] = pred['prediction_Class_9']

In [ ]:
sub_sample.head()

In [ ]:
sub_sample = sub_sample.reset_index()

In [ ]:
sub_sample.to_csv('submission.csv',index=False)